Weather data comes from swish analytics https://swishanalytics.com/mlb/weather?date=2022-11-05

Helpful website for understanding tables: https://medium.com/geekculture/web-scraping-tables-in-python-using-beautiful-soup-8bbc31c5803e

In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2 
import io

Read in game dates from GameLogs.csv

In [6]:
columns = ['Date','VisitingTeam','HomeTeam']
dates = pd.read_csv('source_files/gameLogsFixed.csv', skipinitialspace=True, usecols=columns)
dates['Date'] = pd.to_datetime(dates['Date'])
dates = dates[dates['Date'] > '2020-08-06'] # I went to the website and this is the first day with the weather data
dates['Date'] = dates['Date'].astype(str)
gameDays =  pd.Series(dates['Date']).drop_duplicates().tolist() # drop duplicates

#### Create List of URLs ####
swishAnalytics = 'https://swishanalytics.com/mlb/weather?date='
urlList = []
for games in gameDays:
    urlList.append(swishAnalytics + games)

In [8]:
# created urlList2 so I could run the webscraper in date chunks
urlList2 = urlList

# set up dataframe
website = []
date = []
startTime = [] # time game started in military time (ET)
visiting = []
home = []
avgTemp = [] # avg feels like
totalPrecip = [] # total % precipitation during the game
avgWindSpeed = []

headers = {"Accept-Language": "en-US, en;q=0.5"}

for url in urlList2:
        gameIsNA = 0
        results = requests.get(url, headers=headers)
        soup = BeautifulSoup(results.text,"html.parser")
        weather_div = soup.find_all('div', class_ = 'weather-card')

        ### Loop through each game played that day ###
        for container in weather_div:
                #### Website Link ####
                website.append(url)

                #### Date ####
                date.append(url[-10:])

                #### Team Names ####
                lineup = container.h4.text
                # Visiting #
                team1 = lineup[:lineup.index('@')]
                visiting.append(team1)
                # Home #
                team2 = lineup[lineup.index('@')+2:]
                home.append(team2)

                #### Time ####
                time_div = soup.find('div', class_ = 'col-md-3 pad-5')
                time = container.small.text.strip(' ET')
                meridian = time[-2:] # "AM" or "PM"
                time_without_meridian = time[:-2]
                
                # if there is no weater data append NA for all columns
                if time == 'BA':
                        startTime.append('NA')
                        avgTemp.append('NA')
                        totalPrecip.append('NA')
                        avgWindSpeed.append('NA')
                        continue
                
                hour = int(time[:(time.index(':'))])
                if meridian == "PM" and hour != 12:
                        hour += 12
                        time = str(hour) + time_without_meridian[time.index(':'):]
                elif meridian == "AM" and hour ==12:
                        time = '0' + time_without_meridian[time.index(':'):]
                else: # for AM and noon
                        time = time_without_meridian
                startTime.append(time)

        #### Weather ####
        # Loop through all weather cards #
        temp_table = soup.find_all('table', class_ = 'table table-bordered')
        for table in temp_table:
                # This itterates through the table by row #  
                for row in table.tbody.find_all('tr'):
                        columns = row.find_all('td')
                        if(columns != []):
                                rowName = columns[0].text
                                ## Feels Like ##
                                if rowName == 'Feels Like':
                                        feelsLikeList = []
                                        for hour in columns:
                                                feelsLikeList.append(hour.text.strip('.°'))
                                        if feelsLikeList[1] == 'N/A':
                                                continue
                                        feelsLikeList.remove('N/A')
                                        avgFeelsLike = np.mean([int(i) for i in feelsLikeList[1:]])
                                        avgTemp.append(avgFeelsLike)

                                ## Precipitation ##
                                if rowName == 'Precip %':
                                        precipList = []
                                        for hour in columns:
                                                precipList.append(hour.text.strip('%'))
                                        if precipList[1] == 'N/A':
                                                continue
                                        precipList.remove('N/A')
                                        sumPrecip = np.sum([int(i) for i in precipList[1:]]) / 100
                                        totalPrecip.append(sumPrecip)

                                ## Wind ##
                                if rowName == 'Wind Speed':
                                        windList = []
                                        for hour in columns:
                                                windList.append(hour.text.strip(' mph'))
                                        if windList[1] == 'N/A':
                                                continue
                                        windList.remove('N/A')
                                        avgWind = np.mean([int(i) for i in windList[1:]])
                                        avgWindSpeed.append(avgWind)

weather_df = pd.DataFrame({
'url': website,
'date': date,
'homeTeam': home,
'visitingTeam':visiting,
'gameTime':startTime,
'avgTemp':avgTemp,
'percipitation':totalPrecip,
'avgWindSpeed':avgWindSpeed
})


# saving the dataframe
csv_name = 'weather' + str(index1) + '_' + str(index2) + '.csv'
weather_df.to_csv((csv_name), header=True, index=False, mode='w+')

ValueError: invalid literal for int() with base 10: 'N/A'